In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize, MinMaxScaler

In [ ]:
file_path = "movies_dataset.csv"
df = pd.read_csv(file_path)
df = df.dropna(
    subset=[
        "title",
        "overview",
        "genres",
        "keywords",
        "popularity",
        "poster_path",
        "vote_average",
        "vote_count",
        "release_date",
    ]
)

mask = ~(
    df['genres'].str.contains('Documentary', case=False, na=False) |
    df['genres'].str.contains('porn', case=False, na=False) |
    df['keywords'].str.contains('porn', case=False, na=False) |
    df['keywords'].str.contains('softcore', case=False, na=False) |
    df['keywords'].str.contains('bdsm', case=False, na=False)    
)
df = df[mask].reset_index(drop=True)


Dataset loaded with 159742 rows and 20 columns.
Filtered dataset now has 128924 rows.


In [4]:
filtered_csv_path = "filtered_movies_dataset.csv"
df.to_csv(filtered_csv_path, index=False)
print(f"Filtrelenmiş veri seti '{filtered_csv_path}' dosyasına kaydedildi.")

Filtrelenmiş veri seti 'filtered_movies_dataset.csv' dosyasına kaydedildi.


In [5]:
df["combined"] = df["genres"].fillna('') + " " + df["keywords"].fillna('') + " " + df["overview"].fillna('')

In [ ]:
# TF-IDF (Term Frequency-Inverse Document Frequency): Kelimelerin önemini belirleyip metni vektörleştirir.
# max_features=5000: En sık geçen 5000 kelimeyi dikkate alır.
# fit_transform: Metni vektörlere dönüştürür.
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(df['combined'])

# Sayısal sütunları alalım
numeric_cols = ["vote_average", "vote_count", "popularity", "runtime"]
numeric_features_raw = df[numeric_cols].fillna(0).values

# Sayısal özellikleri ölçeklendirelim
scaler = MinMaxScaler()
numeric_features_scaled = scaler.fit_transform(numeric_features_raw)

from scipy.sparse import hstack
# TF-IDF matrisi (zaten TfidfVectorizer tarafından normalize edilmiş olabilir)
# ve ÖLÇEKLENDİRİLMİŞ sayısal özellikleri birleştirelim
X = hstack([tfidf_matrix, numeric_features_scaled])

# Birleştirilmiş özellik matrisini normalize edelim
# Bu, her filmin birleşik özellik vektörünün birim uzunlukta olmasını sağlar,
# böylece kosinüs benzerliği beklendiği gibi çalışır.
X = normalize(X)

# metric='cosine': Kosinüs benzerliği kullanılır (vektörler arasındaki açıya bakar).
# algorithm='brute': Tüm vektörler arasında doğrudan karşılaştırma yapar.
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(X)

NearestNeighbors(algorithm='brute', metric='cosine')

In [7]:
import joblib
import os

# Kaydetmek için klasör oluştur
os.makedirs("model_files", exist_ok=True)

# Tüm gerekli bileşenleri kaydet
joblib.dump(model_knn, "model_files/knn_model.joblib")
joblib.dump(tfidf, "model_files/tfidf_vectorizer.joblib")
joblib.dump(scaler, "model_files/scaler.joblib")

# X matrisini ve DataFrame'i de kaydedelim
joblib.dump(X, "model_files/feature_matrix.joblib")
df[["id", "title", "genres", "keywords", "overview", "vote_average", "vote_count", 
   "popularity", "runtime", "poster_path", "combined", "release_date"]].to_pickle("model_files/movies_df.pkl")

print("Model ve ilgili bileşenler 'model_files' klasörüne kaydedildi.")

Model ve ilgili bileşenler 'model_files' klasörüne kaydedildi.


In [28]:
def average_similarity_score(X, model, top_indices, k=6):
    X_csr = X.tocsr()
    # Sadece seçilen filmler için KNN uygula
    X_selected = X_csr[top_indices]
    distances, indices = model.kneighbors(X_selected, n_neighbors=k)

    # İlk komşu kendisi olduğundan onu atıyoruz
    scores = 1 - distances[:, 1:]  # cosine distance → similarity
    mean_sim = np.mean(scores)
    return mean_sim

# ✅ 1. Vote average'a göre en yüksek 250 filmi al
top_250_indices = df.sort_values("vote_average", ascending=False).head(250).index

# ✅ 2. Ortalama benzerlik skorunu hesapla
avg_score = average_similarity_score(X, model_knn, top_250_indices, k=6)
print("Ortalama Benzerlik Skoru (Top 250):", round(avg_score, 4))

Ortalama Benzerlik Skoru (Top 250): 0.6584


In [30]:
def recommend_from_two_knn(title1, title2, top_n=5):
    idx1 = df[df["title"] == title1].index[0]
    idx2 = df[df["title"] == title2].index[0]

    # Convert X to CSR format for efficient row slicing
    X_csr = X.tocsr()

    vec1 = X_csr[idx1]
    vec2 = X_csr[idx2]
    combined_vec = (vec1 + vec2) / 2  # ortalama vektör

    # The kneighbors method of NearestNeighbors can work with sparse matrices,
    # so we don't need to change model_knn.kneighbors.
    distances, indices = model_knn.kneighbors(combined_vec, n_neighbors=top_n+2)
    print("Distances:", distances[0])
    print("Indices:", indices[0])
    print("i not in [idx1, idx2] filtrelemesinden önce tüm eşleşmeler:")
    for j, i in enumerate(indices[0]):
        print(df.iloc[i]["title"], "→", 1 - distances[0][j])
    recs = [(df.iloc[i]["title"], round(1 - distances[0][j], 3))
            for j, i in enumerate(indices[0]) if i not in [idx1, idx2]][:top_n]

    return recs

recommend_from_two_knn("Dragon Ball Z: Dead Zone", "Bleach the Movie: Fade to Black")

Distances: [0.17759378 0.17759378 0.39267937 0.40913282 0.41896342 0.42153627
 0.42190227]
Indices: [  2782   2781 111079   1136   2508  68793    467]
i not in [idx1, idx2] filtrelemesinden önce tüm eşleşmeler:
Dragon Ball Z: Dead Zone → 0.8224062192017094
Bleach the Movie: Fade to Black → 0.8224062192017092
The Fixer → 0.6073206268301953
Dragon Ball Z: Lord Slug → 0.5908671771946434
Dragon Ball: Curse of the Blood Rubies → 0.5810365837582266
Militant Eagle → 0.5784637281965418
Dragon Ball Z: Broly – The Legendary Super Saiyan → 0.5780977291615966


[('The Fixer', 0.607),
 ('Dragon Ball Z: Lord Slug', 0.591),
 ('Dragon Ball: Curse of the Blood Rubies', 0.581),
 ('Militant Eagle', 0.578),
 ('Dragon Ball Z: Broly – The Legendary Super Saiyan', 0.578)]